In [22]:
import requests, re
from requests_html import HTMLSession

In [23]:
r=requests.get("https://www.propertypro.ng/property-for-rent?search=surulere")
c=r.content

In [24]:
session = HTMLSession()
r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/surulere')

In [25]:
properties = r.html.find('div.single-room-text')
properties

[<Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>,
 <Element 'div' class=('single-room-text',)>]

In [27]:
properties[8].find('div.fur-areea')[0].text.split()

['3', 'beds', '3', 'baths', '4', 'Toilets']

In [19]:
items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
listings = 20
page_nr = int(items/listings)
page_nr

277

In [20]:
l = []
LP1= "lekki/lekki-phase-1"
locations = ["gbagada"]

for place in locations:

    r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place)

    items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
    listings = 20
    pages = int(items/listings) + 1

    for page in range(pages):

        r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"?page="+str(page))
        properties = r.html.find('div.single-room-text')
        #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
     
        for i in list(range(0,len(properties))):
            d={}
            d["location"] = place
            try:
                d["specific_location"] = properties[i].find('h4')[0].text
            except (IndexError,TypeError,AttributeError):
                d["location"] = None
            try:
                d["features"]= properties[i].find('div.fur-areea')[0].text 
            except (AttributeError,IndexError) as e:
                d["features"]= None
            try:
                d["bed"]= int(re.findall("..bed",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["bed"]= 0 
            try:
                d["bath"]= int(re.findall("..bath",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["bath"]= 0
            try:
                d["toilet"]= int(re.findall("..Toilet",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
            except (IndexError,TypeError,AttributeError,ValueError) as e:
                d["toilet"]= 0
            try:
                d["description"]= properties[i].find('h3.listings-property-title2')[0].text
            except (IndexError,TypeError,AttributeError) as e:
                d["description"]= None

            try:
                d["price"]= properties[i].find('h3.listings-price')[0].text.replace("₦ ","").replace(",","")
            except (IndexError,TypeError,AttributeError) as e:
                d["price"] = None
            l.append(d)

In [21]:
import pandas as pd
ld = pd.DataFrame(l)
ld

,location,specific_location,features,bed,bath,toilet,description,price
0,gbagada,"Diya Street, Ifako GbagadaIfako Gbagada Gbagad...",beds baths Toilets,0,0,0,Open Plan office space,1500000
1,gbagada,"Diya Street, Ifako GbagadaIfako Gbagada Gbagad...",beds baths 1 Toilets,0,0,1,Open Plan office space,1500000
2,gbagada,Phase 1Phase 1 Gbagada Lagos,4 beds 4 baths 4 Toilets,4,4,4,Tastefully Finished 4Bedroom Terraced Duplex i...,65000000
3,gbagada,Sawmill Gbagada Ifako Gbagada Gbagada Lagos,2 beds 2 baths 3 Toilets,2,2,3,5 units of Newly built 2 bedrooms up n ground ...,1200000
4,gbagada,Ifako Gbagada Gbagada Lagos,2 beds 2 baths 3 Toilets,2,2,3,Newly built executive 2bedroom duplex with mod...,1500000
...,...,...,...,...,...,...,...,...
5817,gbagada,"no 6 fatile ayoren street,Gbagada Gbagada Lagos",5 beds 4 baths 4 Toilets,5,4,4,5 Bedroom Detached duplex,3500000
5818,gbagada,"Onasanya mini estate,Gbagada Gbagada Lagos",3 beds 2 baths 2 Toilets,3,2,2,2 bedroom flat,1000000
5819,gbagada,"ifako junction,Gbagada Gbagada Lagos",3 beds 3 baths 4 Toilets,3,3,4,3 bedroom flat,1200000
5820,gbagada,"Ajayi Aina Street, Ifako Barracks Bustop,Gbaga...",3 beds 2 baths 2 Toilets,3,2,2,3 Bedroom Flat Office Space/Residential,800000


In [28]:
l = []
# LP1= "lekki/lekki-phase-1"
locations = ["lekki"]
property_area = ["lekki-phase-1"]
for place in locations:

    r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place)

    for area in property_area:
        
        r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area)

        items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
        listings = 20
        pages = int(items/listings) + 1

        for page in range(pages):

            r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area+"?page="+str(page))
            properties = r.html.find('div.single-room-text')
            #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
        
            for i in list(range(0,len(properties))):
                d={}
                d["location"] = place
                d["property_area"] = area
                try:
                    d["street"] = properties[i].find('h4')[0].text
                except (IndexError,TypeError,AttributeError):
                    d["location"] = None
                try:
                    d["features"]= properties[i].find('div.fur-areea')[0].text 
                except (AttributeError,IndexError) as e:
                    d["features"]= None
                try:
                    d["bed"]= int(re.findall("..bed",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bed"]= 0 
                try:
                    d["bath"]= int(re.findall("..bath",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bath"]= 0
                try:
                    d["toilet"]= int(re.findall("..Toilet",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["toilet"]= 0
                try:
                    d["description"]= properties[i].find('h3.listings-property-title2')[0].text
                except (IndexError,TypeError,AttributeError) as e:
                    d["description"]= None

                try:
                    d["price"]= properties[i].find('h3.listings-price')[0].text.replace("₦ ","").replace(",","")
                except (IndexError,TypeError,AttributeError) as e:
                    d["price"] = None
                l.append(d)

In [29]:
ld = pd.DataFrame(l)
ld

,location,property_area,street,features,bed,bath,toilet,description,price
0,lekki,lekki-phase-1,off Freedom WayLekki Phase 1 Lekki Lagos,beds baths Toilets,0,0,0,A Massive Serviced Duplex with 48 office seat ...,50000
1,lekki,lekki-phase-1,Lekki phase Lekki LagosLekki Phase 1 Lekki Lagos,3 beds 3 baths 4 Toilets,3,3,4,3 Bedroom semi detached duplex,3500000
2,lekki,lekki-phase-1,Oniru EstateLekki Phase 1 Lekki Lagos,0 beds 0 baths 0 Toilets,0,0,0,commercial property,35000
3,lekki,lekki-phase-1,Oniru EstateLekki Phase 1 Lekki Lagos,4 beds 4 baths 5 Toilets,4,4,5,"Newly Built, Tastefully Finished Four (4) Bedr...",5000000
4,lekki,lekki-phase-1,Lekki Phase 1 Lekki Lagos,3 beds 3 baths 4 Toilets,3,3,4,Decent 3-Bedroom Apartment with Swimming Pool ...,2600000
...,...,...,...,...,...,...,...,...,...
7441,lekki,lekki-phase-1,lekki rightLekki Phase 1 Lekki Lagos,2 beds 2 baths 3 Toilets,2,2,3,2 and 3 bedroom apartment,4000000
7442,lekki,lekki-phase-1,Off Hakeem DicksonLekki Phase 1 Lekki Lagos,4 beds 4 baths 4 Toilets,4,4,4,4 bedroom Duplex with a room BQ,3500000
7443,lekki,lekki-phase-1,Victoria Island Lagos,5 beds 5 baths Toilets,5,5,0,Luxury Smart 5bedroom duplex,None
7444,lekki,lekki-phase-1,Cottage DriveLekki Phase 1 Lekki Lagos,3 beds 3 baths 4 Toilets,3,3,4,3 Bedroom Flat,3500000


In [31]:
ld.to_csv("House_prices_lekki_phase1.csv")

In [32]:
l = []

locations = ["yaba"]
property_area = ["abule-ijesha", "abule-oja", "adekunle", "akoka", "alagomeji", "ebute-metta", "fola-agoro", "iwaya", "jibowu", "onike", "sabo", "tejuosho", "yaba"]
for place in locations:

    r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place)

    for area in property_area:
        
        r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area)

        items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
        listings = 20
        pages = int(items/listings) + 1

        for page in range(pages):

            r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area+"?page="+str(page))
            properties = r.html.find('div.single-room-text')
            #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
        
            for i in list(range(0,len(properties))):
                d={}
                d["location"] = place
                d["property_area"] = area
                try:
                    d["street"] = properties[i].find('h4')[0].text
                except (IndexError,TypeError,AttributeError):
                    d["location"] = None
                try:
                    d["features"]= properties[i].find('div.fur-areea')[0].text 
                except (AttributeError,IndexError) as e:
                    d["features"]= None
                try:
                    d["bed"]= int(re.findall("..bed",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bed"]= 0 
                try:
                    d["bath"]= int(re.findall("..bath",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bath"]= 0
                try:
                    d["toilet"]= int(re.findall("..Toilet",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["toilet"]= 0
                try:
                    d["description"]= properties[i].find('h3.listings-property-title2')[0].text
                except (IndexError,TypeError,AttributeError) as e:
                    d["description"]= None

                try:
                    d["price"]= properties[i].find('h3.listings-price')[0].text.replace("₦ ","").replace(",","")
                except (IndexError,TypeError,AttributeError) as e:
                    d["price"] = None
                l.append(d)

In [33]:
ld = pd.DataFrame(l)
ld

,location,property_area,street,features,bed,bath,toilet,description,price
0,yaba,abule-ijesha,Abuleijesha yaba down yaba tech Abule Ijesha Y...,2 beds 2 baths 2 Toilets,2,2,2,2bedroom apartment,700000
1,yaba,abule-ijesha,"Abuleijesha ,close to Morroco Yabatech area Ab...",1 beds 2 baths 2 Toilets,1,2,2,Standard newly built clean Miniflat at Yaba,600000
2,yaba,abule-ijesha,Yabatech BackgateAbule Ijesha Yaba Lagos,1 beds 1 baths 1 Toilets,1,1,1,Newly Built Room self contain At Yabatech Back...,6000000
3,yaba,abule-ijesha,Abule Ijesha Yaba Lagos,1 beds 1 baths 1 Toilets,1,1,1,Newly built blocks of mini flat,450000
4,yaba,abule-ijesha,Abule ijeshaAbule Ijesha Yaba Lagos,2 beds 2 baths 3 Toilets,2,2,3,Standard newly built 2 bedroom flat,700000
...,...,...,...,...,...,...,...,...,...
17534,yaba,yaba,Oshoroko Before Lacampaigne TropicanaIse Town ...,beds baths Toilets,0,0,0,Land With CofO in Ibeju Lekki,3300000
17535,yaba,yaba,Second TollgateLekki Lagos,4 beds 4 baths 5 Toilets,4,4,5,Brand New 4 Bedroom Semi Detached Duplex;,57000000
17536,yaba,yaba,-Old Ikoyi Ikoyi Lagos,4 beds 4 baths 5 Toilets,4,4,5,PRINCE ADEMOLA OSINUGA OFFERS! NEWLY BUILT 4 B...,449999999
17537,yaba,yaba,Ajah Lagos,5 beds 6 baths 7 Toilets,5,6,7,Charming Newly Built 5 Bedroom Detached House ...,75000000


In [34]:
ld.to_csv("House_prices_yaba.csv")

In [37]:
l = []
locations = ["surulere"]
property_area = ["abraham-adesanya"]
for place in locations:

    r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place)

    for area in property_area:
        
        r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area)

        items = int(re.findall("\d+",r.html.find('div.property-sale-number')[0].text)[2])
        listings = 20
        pages = int(items/listings) + 1

        for page in range(pages):

            r = session.get(f'https://www.propertypro.ng/property-for-rent/in/lagos/'+place+"/"+area+"?page="+str(page))
            properties = r.html.find('div.single-room-text')
            #base_url="https://www.propertypro.ng/property-for-rent?search="+place+ "&auto=&type=&bedroom=&max_price="
        
            for i in list(range(0,len(properties))):
                d={}
                d["location"] = place
                d["property_area"] = area
                try:
                    d["street"] = properties[i].find('h4')[0].text
                except (IndexError,TypeError,AttributeError):
                    d["location"] = None
                try:
                    d["features"]= properties[i].find('div.fur-areea')[0].text 
                except (AttributeError,IndexError) as e:
                    d["features"]= None
                try:
                    d["bed"]= int(re.findall("..bed",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bed"]= 0 
                try:
                    d["bath"]= int(re.findall("..bath",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["bath"]= 0
                try:
                    d["toilet"]= int(re.findall("..Toilet",properties[i].find('div.fur-areea')[0].text)[0].strip()[0][0])
                except (IndexError,TypeError,AttributeError,ValueError) as e:
                    d["toilet"]= 0
                try:
                    d["description"]= properties[i].find('h3.listings-property-title2')[0].text
                except (IndexError,TypeError,AttributeError) as e:
                    d["description"]= None

                try:
                    d["price"]= properties[i].find('h3.listings-price')[0].text.replace("₦ ","").replace(",","")
                except (IndexError,TypeError,AttributeError) as e:
                    d["price"] = None
                l.append(d)

In [38]:
ld = pd.DataFrame(l)
ld

,location,property_area,street,features,bed,bath,toilet,description,price
0,surulere,abraham-adesanya,Abraham AdesanyaAbraham Adesanya Surulere Lagos,3 beds 4 baths 4 Toilets,3,4,4,"Three Bedroom Flat for Rent in Surulere, All R...",800000
1,surulere,abraham-adesanya,main streetAbraham Adesanya Surulere Lagos,beds baths Toilets,0,0,0,web designs,50000
2,surulere,abraham-adesanya,-Abraham Adesanya Surulere Lagos,2 beds 1 baths 1 Toilets,2,1,1,2 bedroom flat,1200000
3,surulere,abraham-adesanya,Abraham adesanya estateAbraham Adesanya Surule...,3 beds 3 baths 3 Toilets,3,3,3,Lovely 3 bedroom bungalow,1500000
4,surulere,abraham-adesanya,Tafawa Balewa crescent Abraham Adesanya Surule...,1 beds 0 baths 1 Toilets,1,0,1,A Room Office Space,300000
